In [1]:
# Perform Google Colab installs (if running in Google Colab)
import os

if "COLAB_GPU" in os.environ:
    print("[INFO] Running in Google Colab, installing requirements.")
    #!pip install -U torch # requires torch 2.1.1+ (for efficient sdpa implementation)
    !pip install PyMuPDF # for reading PDFs with Python
    !pip install tqdm # for progress bars
    !pip install sentence-transformers # for embedding models
    !pip install accelerate # for quantization model loading
    !pip install bitsandbytes # for quantizing models (less storage space)
    !pip install flash-attn --no-build-isolation # for faster attention mechanism = faster LLM inference


[INFO] Running in Google Colab, installing requirements.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 115.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 93.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for flash-attn
  Running setup.py clean for flash-attn
Failed to build flash-attn
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (flash-attn)


In [2]:
!pip uninstall -y torch torchvision torchaudio transformers sentence-transformers

!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

!pip install -U transformers sentence-transformers


Found existing installation: torch 2.9.0+cu126
Uninstalling torch-2.9.0+cu126:
  Successfully uninstalled torch-2.9.0+cu126
Found existing installation: torchvision 0.24.0+cu126
Uninstalling torchvision-0.24.0+cu126:
  Successfully uninstalled torchvision-0.24.0+cu126
Found existing installation: torchaudio 2.9.0+cu126
Uninstalling torchaudio-2.9.0+cu126:
  Successfully uninstalled torchaudio-2.9.0+cu126
Found existing installation: transformers 4.57.1
Uninstalling transformers-4.57.1:
  Successfully uninstalled transformers-4.57.1
Found existing installation: sentence-transformers 5.1.2
Uninstalling sentence-transformers-5.1.2:
  Successfully uninstalled sentence-transformers-5.1.2
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 780.4/780.4 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 133.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 107.4 MB/s eta 0:00:00
     ━━━

In [3]:
# Download PDF file
import os
import requests

# Get PDF document
pdf_path = "human-nutrition-text.pdf"

# Download PDF if it doesn't already exist
if not os.path.exists(pdf_path):
    print("File doesn't exist, downloading...")

    # The URL of the PDF you want to download
    url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"

    # The local filename to save the downloaded file
    filename = pdf_path

    # Send a GET request to the URL
    response = requests.get(url)

    # Check if the request was successful
    if response.status_code == 200:
        # Open a file in binary write mode and save the content to it
        with open(filename, "wb") as file:
            file.write(response.content)
        print(f"The file has been downloaded and saved as {filename}")
    else:
        print(f"Failed to download the file. Status code: {response.status_code}")
else:
    print(f"File {pdf_path} exists.")


File doesn't exist, downloading...
The file has been downloaded and saved as human-nutrition-text.pdf


In [10]:
import fitz # This is the PyMuPdf
from tqdm.auto import tqdm # This Helps in faster iteration
def text_formatter(text: str) ->str:
  #Performs Minor operation on the text
  cleaned_text = text.replace("\n"," ").strip()
  return cleaned_text

# Now we will open the pdf
def open_and_read_pdf(pdf_path: str) ->list:
  doc = fitz.open(pdf_path) # opening a document
  pages_texts = [] # this is where we store the text contained in the certain number of page
  for page_number, page in tqdm(enumerate(doc)):
    text = page.get_text()
    text = text_formatter(text)
    pages_texts.append({"page_numeber": page_number + 1,
                        "page_char_count": len(text),
                        "page_word_count": len(text.split(" ")),
                        "page_sentence_count": len(text.split(". ")),
                        "Page_token_count": len(text)/4,
                        "text": text
                        })
  return pages_texts # Moved return statement outside the loop

pages_texts = open_and_read_pdf(pdf_path = pdf_path)
pages_texts[:3]

0it [00:00, ?it/s]

[{'page_numeber': 1,
  'page_char_count': 29,
  'page_word_count': 4,
  'page_sentence_count': 1,
  'Page_token_count': 7.25,
  'text': 'Human Nutrition: 2020 Edition'},
 {'page_numeber': 2,
  'page_char_count': 0,
  'page_word_count': 1,
  'page_sentence_count': 1,
  'Page_token_count': 0.0,
  'text': ''},
 {'page_numeber': 3,
  'page_char_count': 320,
  'page_word_count': 54,
  'page_sentence_count': 1,
  'Page_token_count': 80.0,
  'text': 'Human Nutrition: 2020  Edition  UNIVERSITY OF HAWAI‘I AT MĀNOA  FOOD SCIENCE AND HUMAN  NUTRITION PROGRAM  ALAN TITCHENAL, SKYLAR HARA,  NOEMI ARCEO CAACBAY, WILLIAM  MEINKE-LAU, YA-YUN YANG, MARIE  KAINOA FIALKOWSKI REVILLA,  JENNIFER DRAPER, GEMADY  LANGFELDER, CHERYL GIBBY, CHYNA  NICOLE CHUN, AND ALLISON  CALABRESE'}]

In [11]:
import pandas as pd
df = pd.DataFrame(pages_texts)
df.head()

,page_numeber,page_char_count,page_word_count,page_sentence_count,Page_token_count,text
0,1,29,4,1,7.25,Human Nutrition: 2020 Edition
1,2,0,1,1,0.00,
2,3,320,54,1,80.00,Human Nutrition: 2020 Edition UNIVERSITY OF ...
3,4,212,32,1,53.00,Human Nutrition: 2020 Edition by University of...
4,5,797,145,2,199.25,Contents Preface University of Hawai‘i at Mā...


In [12]:
df.describe().round(2)

,page_numeber,page_char_count,page_word_count,page_sentence_count,Page_token_count
count,1208.00,1208.00,1208.00,1208.00,1208.00
mean,604.50,1148.00,198.30,9.97,287.00
std,348.86,560.38,95.76,6.19,140.10
min,1.00,0.00,1.00,1.00,0.00
25%,302.75,762.00,134.00,4.00,190.50
50%,604.50,1231.50,214.50,10.00,307.88
75%,906.25,1603.50,271.00,14.00,400.88
max,1208.00,2308.00,429.00,32.00,577.00
